In [1]:
from semantic_kernel.connectors.memory.chroma import ChromaMemoryStore
import os

agent_name = "OpenAI"
memories_dir = f"{os.getcwd()}/agents/{agent_name}/memories"

chroma_client = ChromaMemoryStore(
    persist_directory=memories_dir,
)

memories_exist = await chroma_client.does_collection_exist_async("memories")
if not memories_exist:
    await chroma_client.create_collection_async(collection_name="memories")
    memories = await chroma_client.get_collection_async(collection_name="memories")
else:
    memories = await chroma_client.get_collection_async(collection_name="memories")


Using embedded DuckDB with persistence: data will be stored in: /home/josh/josh/Repos/AGiXT/agixt/agents/OpenAI/memories


In [5]:
await chroma_client.does_collection_exist_async("memories")


False

In [4]:
from hashlib import sha256
from Embedding import Embedding
from datetime import datetime
from semantic_kernel.connectors.memory.chroma import ChromaMemoryStore
from semantic_kernel.memory.memory_record import MemoryRecord
from Agent import Agent

description = "Test memory"
external_source_name = "Test memory"
content = "Test memory"
agent_name = "OpenAI"
agent_config = Agent(agent_name=agent_name).agent_config
memories_dir = f"{os.getcwd()}/agents/{agent_name}/memories"
chroma_client = ChromaMemoryStore(
    persist_directory=memories_dir,
)
embedder, chunk_size = await Embedding(AGENT_CONFIG=agent_config).get_embedder()
record = MemoryRecord(
    is_reference=False,
    id=sha256((content + datetime.now().isoformat()).encode()).hexdigest(),
    text=content,
    timestamp=datetime.now().isoformat(),
    description=description,
    external_source_name=external_source_name,  # URL or File path
    embedding=await embedder(content),
)

await chroma_client.upsert_async(
    collection_name="memories",
    record=record,
)

2023-05-31 07:50:04,387 | INFO | Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
2023-05-31 07:50:04,387 | INFO | Running Chroma using direct local API.
2023-05-31 07:50:04,388 | WARNING | Using embedded DuckDB with persistence: data will be stored in: /home/josh/josh/Repos/AGiXT/agixt/agents/OpenAI/memories
2023-05-31 07:50:04,396 | INFO | No existing DB found in /home/josh/josh/Repos/AGiXT/agixt/agents/OpenAI/memories, skipping load
2023-05-31 07:50:04,397 | INFO | No existing DB found in /home/josh/josh/Repos/AGiXT/agixt/agents/OpenAI/memories, skipping load
2023-05-31 07:50:05,217 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=385 request_id=525b5cb57bfa8e679705074510af0042 response_code=200


Exception: Collection 'memories' does not exist

In [ ]:
# Test get memories
from Agent import Agent
from Memories import Memories

content = "Test memory"
agent_name = "OpenAI"
agent_config = Agent(agent_name=agent_name).agent_config
memories = Memories(agent_name=agent_name, agent_config=agent_config)
await memories.get_memories()
await memories.context_agent(query=content, top_results_num=2)

In [ ]:
# Store a memory
await memories.store_memory(content="Test memory", description="Test memory")

In [6]:
await chroma_client.create_collection_async(collection_name="memories")


In [2]:
collection = await chroma_client.get_collection_async(collection_name="memories")


In [3]:
collection.json()

'{"name": "memories", "id": "7df28658-812e-427b-b1f7-7e6255fd56af", "metadata": null}'